## Get started with TensorBoard

In machine learning, to improve something you often need to be able to measure it. TensorBoard is a tool for providing the measurements and visualizations needed during the machine learning workflow. It enables tracking experiment metrics like loss and accuracy, visualizing the model graph, projecting embeddings to a lower dimensional space, and much more.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
# Clear any logs from previous runs
!rm -rf ./logs/

'rm' is not recognized as an internal or external command,
operable program or batch file.


Using the MNIST dataset as the example, normalize the data and write a function that creates a simple Keras model for classifying the images into 10 classes.

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

Using TensorBoard with Keras Model.fit()

When training with Keras's Model.fit(), adding the tf.keras.callbacks.TensorBoard callback ensures that logs are created and stored. Additionally, enable histogram computation every epoch with histogram_freq=1 (this is off by default)

Place the logs in a timestamped subdirectory to allow easy selection of different training runs.

### Using TensorBoard with Keras Model.fit()
When training with Keras's Model.fit(), adding the tf.keras.callbacks.TensorBoard callback ensures that logs are created and stored. Additionally, enable histogram computation every epoch with histogram_freq=1 (this is off by default)

Place the logs in a timestamped subdirectory to allow easy selection of different training runs.

In [6]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

c:\Users\User\miniconda3\envs\tf_ml\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8941 - loss: 0.3590 - val_accuracy: 0.9691 - val_loss: 0.1025
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9694 - loss: 0.1005 - val_accuracy: 0.9744 - val_loss: 0.0819
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9789 - loss: 0.0682 - val_accuracy: 0.9771 - val_loss: 0.0703
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9850 - loss: 0.0507 - val_accuracy: 0.9750 - val_loss: 0.0794
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9868 - loss: 0.0407 - val_accuracy: 0.9808 - val_loss: 0.0666


Start TensorBoard through the command line or within a notebook experience. The two interfaces are generally the same. In notebooks, use the %tensorboard line magic. On the command line, run the same command without "%".

In [7]:
%tensorboard --logdir logs/fit

In [8]:
tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16904), started 0:00:07 ago. (Use '!kill 16904' to kill it.)

A brief overview of the visualizations created in this example and the dashboards (tabs in top navigation bar) where they can be found:

- **Scalars** show how the loss and metrics change with every epoch. You can use them to also track training    speed, learning rate, and other scalar values. Scalars can be found in the Time Series or Scalars dashboards.
- **Graphs** help you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. Graphs can be found in the Graphs dashboard.
- **Histograms** and Distributions show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way. Histograms can be found in the Time - Series or Histograms dashboards. Distributions can be found in the Distributions dashboard.
  
Additional TensorBoard dashboards are automatically enabled when you log other types of data. For example, the Keras TensorBoard callback lets you log images and embeddings as well. You can see what other dashboards are available in TensorBoard by clicking on the "inactive" dropdown towards the top right.

### Using TensorBoard with other methods

When training with methods such as tf.GradientTape(), use tf.summary to log the required information.

Use the same dataset as above, but convert it to tf.data.Dataset to take advantage of batching capabilities:

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

The training code follows the advanced quickstart tutorial, but shows how to log metrics to TensorBoard. Choose loss and optimizer:

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [11]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

Define the training and test functions:

In [12]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

Set up summary writers to write the summaries to disk in a different logs directory:

In [13]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

Start training. Use tf.summary.scalar() to log metrics (loss and accuracy) during training/testing within the scope of the summary writers to write the summaries to disk. You have control over which metrics to log and how often to do it. Other tf.summary functions enable logging other types of data.

In [22]:
model = create_model()  # Reset model
optimizer = tf.keras.optimizers.Adam()  # Reinitialize optimizer # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss = tf.keras.metrics.Mean()  # Creating a new Mean instance
  test_loss = tf.keras.metrics.Mean()   # This resets it automatically
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

Epoch 1, Loss: 0.24463534355163574, Accuracy: 92.86328887939453, Test Loss: 0.12463685125112534, Test Accuracy: 96.48500061035156
Epoch 2, Loss: 0.10592766106128693, Accuracy: 96.87000274658203, Test Loss: 0.08397915214300156, Test Accuracy: 97.37999725341797
Epoch 3, Loss: 0.07417217642068863, Accuracy: 97.77333068847656, Test Loss: 0.07714098691940308, Test Accuracy: 97.64999389648438
Epoch 4, Loss: 0.0554547943174839, Accuracy: 98.26499938964844, Test Loss: 0.06713751703500748, Test Accuracy: 97.87999725341797
Epoch 5, Loss: 0.042795874178409576, Accuracy: 98.64500427246094, Test Loss: 0.06105022132396698, Test Accuracy: 98.0999984741211


Open TensorBoard again, this time pointing it at the new log directory. We could have also started TensorBoard to monitor training while it progresses.

In [23]:
%tensorboard --logdir logs/gradient_tape

That's it! You have now seen how to use TensorBoard both through the Keras callback and through tf.summary for more custom scenarios.